# Organizing time series metabolomics data (compound concentrations) into an array with dimensions of strain-replicate, time, and compound

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('../Data/20151130_OAM1_QuantResults_v2C.csv')
df

,Sample,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,leucine Results,phenylalanine Results,isoleucine Results,...,threonine Results,glutamine Results,glycine Results,asparagine Results,histidine Results,serine Results,glutamate Results,aspartate Results,lysine Results,arginine Results
0,NaN,NaN,Name,Data File,Type,Level,Acq. Date-Time,Final Conc.,Final Conc.,Final Conc.,...,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.,Final Conc.
1,!,!,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 1:10 PM,98271.1147,105638.0265,104417.5952,...,111760.7773,112814.1930,111788.6120,126826.4872,97834.0787,121143.7200,121906.4412,130784.7968,124630.3555,116730.3451
2,!,!,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 4:33 PM,91986.6477,93925.4729,95888.1950,...,101575.2277,101620.0393,102581.0434,113173.4553,91015.5479,109372.2429,109260.2552,122921.7411,108724.2979,103363.5179
3,NaN,!,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 7:57 PM,98049.6649,102496.3786,103039.1871,...,111343.4842,111244.7480,106746.5409,123644.6403,89985.7446,116935.1761,119284.9482,130160.9927,118825.9913,110217.2571
4,!,!,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 9:15 AM,100311.5929,107149.3256,94865.0484,...,98883.8606,99212.0401,99138.8811,88598.2077,104170.3593,91729.4308,89493.7771,78159.5985,98015.4426,99239.8979
5,!,!,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 12:39 PM,96300.2770,98025.1689,99704.8464,...,93902.4595,92570.3425,86784.4339,80591.4867,92535.2992,82272.8901,83455.2711,71366.0097,85226.3133,89593.9839
6,NaN,!,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 4:02 PM,106085.7998,105480.5946,105393.8471,...,99747.4627,96039.9567,93611.9822,80416.9720,103732.3727,85111.7028,84047.3637,69471.3062,86501.6968,94301.2964
7,NaN,!,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 1:50 PM,49882.6272,40754.3930,47272.0027,...,41477.6227,39718.8732,48853.7139,45747.3875,55551.4870,47553.8270,46988.7424,54124.9276,43822.0930,44712.0573
8,NaN,!,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 5:14 PM,54835.7355,48647.7043,56231.8878,...,49633.9908,52300.7443,54200.8408,56594.5849,58832.4658,56601.1139,58494.7852,64498.1270,52392.6488,53214.4660
9,NaN,!,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 8:37 PM,53120.7852,46674.7361,48191.0521,...,46410.2231,49183.1221,52677.2023,51669.1931,55645.7920,52225.0829,54368.1363,60144.4163,47661.5525,48919.3648


### Clean up the header row so that none of the columns are named "Unnamed", but instead are named something more meaningful

In [3]:
#only run once.  otherwise, you have to reset kernel to avoid a reference error.
new_col = []
for i,k in enumerate(df.keys()):
    if not "Unnamed" in k:
        my_tag = k
    new_col.append('%s %s'%(my_tag ,df.ix[0,k]))
df.columns = new_col
df = df.drop(df.index[[0]])
#and clean up an unnecessary colum
del df['Sample nan']
df

,Sample Name,Sample Data File,Sample Type,Sample Level,Sample Acq. Date-Time,leucine Results Final Conc.,phenylalanine Results Final Conc.,isoleucine Results Final Conc.,tryptophan Results Final Conc.,methionine Results Final Conc.,...,threonine Results Final Conc.,glutamine Results Final Conc.,glycine Results Final Conc.,asparagine Results Final Conc.,histidine Results Final Conc.,serine Results Final Conc.,glutamate Results Final Conc.,aspartate Results Final Conc.,lysine Results Final Conc.,arginine Results Final Conc.
1,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 1:10 PM,98271.1147,105638.0265,104417.5952,110921.3220,101031.2986,...,111760.7773,112814.1930,111788.6120,126826.4872,97834.0787,121143.7200,121906.4412,130784.7968,124630.3555,116730.3451
2,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 4:33 PM,91986.6477,93925.4729,95888.1950,99333.3535,93291.8436,...,101575.2277,101620.0393,102581.0434,113173.4553,91015.5479,109372.2429,109260.2552,122921.7411,108724.2979,103363.5179
3,OAA2_100_1,20151123_SJ_OAM1_6460_OAA2_100_1_HLPos_OAA2_10...,Cal,5,11/25/2015 7:57 PM,98049.6649,102496.3786,103039.1871,97730.5302,99901.4116,...,111343.4842,111244.7480,106746.5409,123644.6403,89985.7446,116935.1761,119284.9482,130160.9927,118825.9913,110217.2571
4,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 9:15 AM,100311.5929,107149.3256,94865.0484,107817.1020,103537.8672,...,98883.8606,99212.0401,99138.8811,88598.2077,104170.3593,91729.4308,89493.7771,78159.5985,98015.4426,99239.8979
5,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 12:39 PM,96300.2770,98025.1689,99704.8464,95766.3438,95514.0620,...,93902.4595,92570.3425,86784.4339,80591.4867,92535.2992,82272.8901,83455.2711,71366.0097,85226.3133,89593.9839
6,OAA2_100_2,20151123_SJ_OAM1_6460_OAA2_100_2_HLPos_OAA2_10...,Cal,5,11/27/2015 4:02 PM,106085.7998,105480.5946,105393.8471,104587.2880,103595.1191,...,99747.4627,96039.9567,93611.9822,80416.9720,103732.3727,85111.7028,84047.3637,69471.3062,86501.6968,94301.2964
7,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 1:50 PM,49882.6272,40754.3930,47272.0027,41126.7764,46244.8647,...,41477.6227,39718.8732,48853.7139,45747.3875,55551.4870,47553.8270,46988.7424,54124.9276,43822.0930,44712.0573
8,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 5:14 PM,54835.7355,48647.7043,56231.8878,50582.9218,52635.5390,...,49633.9908,52300.7443,54200.8408,56594.5849,58832.4658,56601.1139,58494.7852,64498.1270,52392.6488,53214.4660
9,OAA2_50_1,20151123_SJ_OAM1_6460_OAA2_50_1_HLPos_OAA2_50_...,Cal,4,11/25/2015 8:37 PM,53120.7852,46674.7361,48191.0521,46931.9467,52512.0903,...,46410.2231,49183.1221,52677.2023,51669.1931,55645.7920,52225.0829,54368.1363,60144.4163,47661.5525,48919.3648
10,OAA2_50_2,20151123_SJ_OAM1_6460_OAA2_50_2_HLPos_OAA2_50_...,Cal,4,11/27/2015 9:56 AM,48361.7381,42180.5841,41552.4442,40586.6561,46092.1139,...,39492.2578,40054.9731,43925.4006,37843.2713,51537.2350,40154.1725,39333.3902,36179.5504,36226.1648,38833.5934


### Create a list of the compounds measured in each sample file

In [4]:
my_val_keys = []
for k in df.keys():
    if "Final Conc." in k:
        my_val_keys.append(k)

### Extract File, Concentration, Strain, Replicate, and Timepoint information
* Ignoring the calibration curve samples (OAA), the water blank samples (H2O), and uninoculated medium samples (medB), we need to pull the relevant sample information out of the filename

In [5]:
my_files = [] #each data file
my_vals = [] #values for each compound
my_strain = [] #strain name
my_replicate = [] #replicate name
my_strain_replicate = [] #strain-replicate
my_time = [] #time dimension
for i in range(df.shape[0]):
    #OAA means it's a cal-curve
    if not "OAA" in df.iloc[i]['Sample Data File'].split('_')[4]:
        #H20 means it's a water blank
        if not "H2O" in df.iloc[i]['Sample Data File'].split('_')[4]:
            #medB means it's a medium control
            if not "medB" in df.iloc[i]['Sample Data File'].split('_')[4]:
                my_files.append(df.iloc[i]['Sample Data File'])
                temp = df.iloc[i]['Sample Data File'].split('_')[4].split('-')
                my_strain.append(temp[0])
                my_replicate.append(temp[1])
                my_strain_replicate.append('%s %s'%(temp[0],temp[1]))
                my_time.append(float(temp[2].replace('t','')))
                temp_vals = []
                for mvk in my_val_keys:
                    temp_vals.append(df.iloc[i][mvk])
                my_vals.append(temp_vals)

### Make a 3D array to store all information (strain-replicate x concentration x timepoint)

In [6]:
#unique timepoints
u_time = np.unique(my_time)
#unique filenames
u_files = np.unique(my_strain_replicate)

M = np.zeros((len(u_files),len(my_val_keys),len(u_time)))

for i,uf in enumerate(u_files):
    for j,uv in enumerate(my_val_keys):
        for k,ut in enumerate(u_time):
            idx1 = np.argwhere( (np.asarray(my_strain_replicate) == uf) & (np.asarray(my_time) == ut )).flatten()
            idx2 = np.argwhere(np.asarray(my_val_keys) == uv).flatten()
            if len(idx1)>0:
                if np.isnan(float(my_vals[idx1][idx2])):
                    pass
                else:
                    M[i,j,k] = float(my_vals[idx1][idx2])
#dim one is labeled by u_files
#dim two is labeled by my_val_keys
#dim three is labeled by u_time

/Users/Onur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/Onur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


### Final step is to save the data for use in later Python programs

In [7]:
data = {}
data['dim1'] = u_files
data['dim2'] = my_val_keys
data['dim3'] = u_time
data['M'] = M
with open("../Data/metabolomics_data.pkl",'w') as f:
    pickle.dump(data,f)